1\. **Radioactive decay chain**

${\rm Tl}^{208}$ decays to ${\rm Pb}^{208}$ with a half-life $\tau$ of 3.052 minutes. Suppose to start with a sample of 1000 Thallium atoms and 0 of Lead atoms.

* Take steps in time of 1 second and at each time-step decide whether each Tl atom has decayed or not, accordingly to the probability $p(t)=1-2^{-t/\tau}$. Subtract the total number of Tl atoms that decayed at each step from the Tl sample and add them to the Lead one. Plot the evolution of the two sets as a function of time  
* Repeat the exercise by means of the inverse transform method: draw 1000 random numbers from the non-uniform probability distribution $p(t)=2^{-t/\tau}\frac{\ln 2}{\tau}$ to represent the times of decay of the 1000 Tl atoms. Make a plot showing the number of atoms that have not decayed as a function of time

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Constants
tau = 3.052 * 60  # Half-life of Thallium in seconds
total_atoms = 1000

# Part 1: Simulation using probability calculation

# Initialize arrays to store the number of Thallium and Lead atoms at each time step
time_steps = np.arange(0, 301, 1)  # Time steps in seconds (0 to 5 minutes)
thallium_atoms = np.zeros(len(time_steps))
lead_atoms = np.zeros(len(time_steps))

# Set initial conditions
thallium_atoms[0] = total_atoms
lead_atoms[0] = 0

# Simulate the decay process
for i, t in enumerate(time_steps[1:]):
    p = 1 - 2 ** (-t / tau)
    decayed_atoms = np.random.binomial(thallium_atoms[i], p)
    thallium_atoms[i+1] = thallium_atoms[i] - decayed_atoms
    lead_atoms[i+1] = lead_atoms[i] + decayed_atoms

# Plot the results
plt.plot(time_steps, thallium_atoms, label='Thallium')
plt.plot(time_steps, lead_atoms, label='Lead')
plt.xlabel('Time (seconds)')
plt.ylabel('Number of Atoms')
plt.title('Thallium and Lead Decay')
plt.legend()
plt.show()

# Part 2: Simulation using inverse transform method

def inv_decay_cdf(y):
    return - tau * np.log2(1-y)

fig = plt.figure()
x = np.arange(0,2000,1)

# Perform the inverse of the inverse transform method
total_atoms = 1000
athoms_prob = np.random.random(total_atoms)
dacay_time = inv_decay_cdf(athoms_prob)

dacay_time_sorted = np.sort(dacay_time)[::-1]

x = np.arange(0,len(dacay_time_sorted),1)
plt.plot(dacay_time_sorted,x)
plt.title('Times of decay of the 1000 Thallium atoms')
plt.xlabel('Times [s]')
plt.ylabel('Thallium atoms')
plt.show()

2\. **Monte Carlo integration: hit/miss vs mean value method**

Consider the function: 

$$f(x) =\sin^2{\left( \frac{1}{1-x} \right)}$$

* Plot the function and compute the integral of $f(x)$ between 0 and 2 with the hit/miss method. Evaluate the error of your estimate (hint: repeat the integral $N$ times, and from the distribution of the integrals take the mean value and the standard deviation, the latter rescaled by the appropriate factor)
* Repeat the integral with the mean value method. Evaluate the error and compare it with the previous one.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def f(x):
    return (np.sin(1 / (1 - x))) ** 2

# Plot function
test = np.linspace(0, 2, n)
plt.figure()
plt.plot(test, f(test))
plt.xlabel("X")
plt.ylabel("Y = f(X)")
plt.show()

# Calculate integral hit miss
n = 2000
h, a, b = 1, 0, 2
c = 0

for i in range(n):
    y = h * np.random.rand()
    x = (b - a) * np.random.rand()
    if y < f(x):
        c += 1

I = (c / n) * h * (b - a)
print("Using the hit/miss method, the estimated value of the integral is", I)

m = 9
Im = np.zeros(m)

for i in range(m):
    c = 0

    for j in range(n):
        y = h * np.random.rand()
        x = (b - a) * np.random.rand()
        if y < f(x):
            c += 1

    Im[i] = (c / n) * h * (b - a)

I1 = np.sum(Im) / m
I2 = np.sum(Im ** 2) / m
sM = np.sqrt(I2 - I1 ** 2)
print("The estimated error using the hit/miss method is {:.6f}".format(sM))

# Calculate integral mean value
n = 2000
h, a, b = 1, 0, 2
somma = 0

for i in range(n):
    somma += f(h * np.random.rand())

J = ((b - a) / n) * somma
print("Using the mean value method, the estimated value of the integral is", J)

m = 9
Im = np.zeros(m)

for i in range(m):
    c = 0

    for j in range(n):
        y = h * np.random.rand()
        x = (b - a) * np.random.rand()
        if y < f(x):
            c += 1

    Im[i] = (c / n) * h * (b - a)

I1 = np.sum(Im) / m
I2 = np.sum(Im ** 2) / m
mean_sM = np.sqrt(I2 - I1 ** 2)
print("The estimated error using the mean value method is {:.6f}".format(mean_sM))

3\. **Monte Carlo integration in high dimension**

* Compute the area of a circle of unit radius, by integrating the function:

$$
f(x,y)=
\left\{
\begin{array}{ll}
      1 & x^2+y^2\le 1 \\
      0 & {\rm elsewhere}
\end{array} 
\right.
$$

* Generalize the result for a 10D sphere.

In [ ]:
import numpy as np

def monte_carlo_sphere(radius, num_points, dimensions=2):
    points = np.random.uniform(-radius, radius, size=(num_points, dimensions))
    inside_circle = np.linalg.norm(points, axis=1) <= radius
    
    circle_area = np.mean(inside_circle) * ((2 * radius) ** dimensions)
    return circle_area

radius = 1
num_points = 100000

area = monte_carlo_sphere(radius, num_points)
print("The area of the circle is:", area)

md_volume = monte_carlo_sphere(radius, num_points, 10)
print("The multidimensional volume of the 10D sphere is:", md_volume)

4\. **Monte Carlo integration with importance sampling** 

Calculate the value of the integral:

$$
I=\int_0^1 \frac{x^{-1/2}}{e^x+1} dx
$$

using the importance sampling method with $w(x)=1/\sqrt{x}$. You should expect a result around 0.84.

In [ ]:
import numpy as np
from scipy import stats
from scipy.integrate import quad

def f(x):
    return (x**(-1/2)) / (np.exp(x) + 1)

def w(x):
    return 1 / np.sqrt(x)

def importance_sampling_integral(num_samples):
    w_value, err = quad(w, 0, 1)
    
    # Use the exponential distribution
    x = stats.powerlaw.rvs(0.5, size = num_samples)
    
    integral_sum = np.sum(f(x) / w(x))
    integral = w_value/ num_samples * integral_sum
    
    return integral

num_samples = 100000

result = importance_sampling_integral(num_samples)
print("The integral I is:", result)